# Dynamical Analysis of a T-cell Survival Network

In this notebook, we use [AEON](https://biodivine.fi.muni.cz/aeon/) to analyse the dynamical behaviour of a well know Boolean network describing a T-cell survival mechanism which arises in the context of T-LGL (T-cell large granular lymphocyte) leukemia. In individuals suffering from this disease, mature cytotoxic T-lymphocytes (CTLs) exhibit abnormal proliferation capabilities. Normally, CTLs are generated to eliminate cells infected by a virus. While normal CTLs undergo a programmed cell death once the virus is neutralised, leukemic CTLs remain long-term competent \[1]. The cause of this behaviour has been identified as abnormal dysregulation of a few critical components of the signal transduction network responsible for activation of the programmed cell death mechanism \[2].

The actual Boolean network has been first described in \[3], where it was constructed based on extensive literature search. Subsequently, network attractors and response to node pertrubation was explored using analytical methods, simulation, and network reduction in \[4]. It consists of 60 proteins, mRNAs, and small molecules. Here, the network description was obtained from the [CellCollective](https://research.cellcollective.org/?dashboard=true#module/2407:1/tlgl-survival-network-2011/1) database as `t-lgl-survival.sbml`. The primary function of the network is regulated by the `Stimuli` input variable, which corresponds to successful detection of targeted virions. The main output node is `Apoptosis`, which models a successful initiation of the programmed cell death. Aside from `Stimuli`, there are five other input entities (i.e. logical parameters) which influence the behaviour of the network. Similarly, aside from `Apoptosis`, the network also tracks expression of `Proliferation` and `Cytoskeleton_signaling` phenotypes.

As opposed to \[4], here, we analyse the network in full (i.e. without state space reduction). Our first goal is to reproduce the analysis performed in \[4]. Subsequently, we will attempt to add additional insight into the network behaviour by extending the analysis beyond the settings considered in \[4].

### Replication study

The first step is to recreate the test conditions as examined in \[4]. This means loading the model and setting `Stimuli = IL15 = PDGF = true`, and `Stimuli2 = CD45 = TAX = false`. This model has no remaining free parameters (and conseuqently, only one colour). As a result, we can quite easily compute its attractors, as well as their basins of attraction.

In [1]:
from biodivine_aeon import *

In [2]:
model = BooleanNetwork.from_file('t-lgl-survival.sbml')

# Fix input variables according to [4]
model.set_update_function("Stimuli", "true")
model.set_update_function("IL15", "true")
model.set_update_function("PDGF", "true")
model.set_update_function("Stimuli2", "false")
model.set_update_function("CD45", "false")
model.set_update_function("TAX", "false")

graph = SymbolicAsyncGraph(model)

In [3]:
attractors = find_attractors(graph)

print("Attractors:", len(attractors))

A1 = attractors[0]
A2 = attractors[1]
A3 = attractors[2]

print("A1:", A1.vertices().cardinality())
print("A2:", A2.vertices().cardinality())
print("A3:", A3.vertices().cardinality())

Attractors: 3
A1: 1.0
A2: 4.0
A3: 4.0


First claim that we'd like to verify is about the structure of the network attractors. According to \[4], we should see a stable attractor with `Apoptosis = true`, and two unstable attractors with `Apoptosis = false`. Here, the instability when `Apoptosis = false` is caused by a feedback loop between `CTLA4` and `TCR`, while the existence of two independent unstable attractors is caused by a bi-stable regime of the variable `P2`, which otherwise plays no major role in the structure of the attractors.

In [4]:
apoptosis = graph.fix_variable("Apoptosis", True)
proliferation = graph.fix_variable("Proliferation", True)
cytoskeleton = graph.fix_variable("Cytoskeleton_signaling", True)
tcr = graph.fix_variable("TCR", True)
ctla4 = graph.fix_variable("CTLA4", True)
p2 = graph.fix_variable("P2", True)

print("==== A1 Phenotypes ====")
print("A1[Apoptosis] is always True:", A1.minus(apoptosis).is_empty())
print("A1[Proliferation] is always False:", A1.intersect(proliferation).is_empty())
print("A1[Cytoskeleton] is always False:", A1.intersect(cytoskeleton).is_empty())

print("==== A2 Phenotypes ====")
print("A2[Apoptosis] is always False:", A2.intersect(apoptosis).is_empty())
print("A2[Proliferation] is always False:", A2.intersect(proliferation).is_empty())
print("A2[Cytoskeleton] is always True:", A2.minus(cytoskeleton).is_empty())
print("A2[TCR] is always True:", A2.minus(tcr).is_empty())
print("A2[TCR] is always False:", A2.intersect(tcr).is_empty())
print("A2[CTLA4] is always True:", A2.minus(ctla4).is_empty())
print("A2[CTLA4] is always False:", A2.intersect(ctla4).is_empty())
print("A2[P2] is always False:", A2.intersect(p2).is_empty())

print("==== A3 Phenotypes ====")
print("A3[Apoptosis] is always False:", A3.intersect(apoptosis).is_empty())
print("A3[Proliferation] is always False:", A3.intersect(proliferation).is_empty())
print("A3[Cytoskeleton] is always True:", A3.minus(cytoskeleton).is_empty())
print("A3[CTLA4] is always True:", A3.minus(ctla4).is_empty())
print("A3[CTLA4] is always False:", A3.intersect(ctla4).is_empty())
print("A3[P2] is always False:", A3.minus(p2).is_empty())

==== A1 Phenotypes ====
A1[Apoptosis] is always True: True
A1[Proliferation] is always False: True
A1[Cytoskeleton] is always False: True
==== A2 Phenotypes ====
A2[Apoptosis] is always False: True
A2[Proliferation] is always False: True
A2[Cytoskeleton] is always True: True
A2[TCR] is always True: False
A2[TCR] is always False: False
A2[CTLA4] is always True: False
A2[CTLA4] is always False: False
A2[P2] is always False: True
==== A3 Phenotypes ====
A3[Apoptosis] is always False: True
A3[Proliferation] is always False: True
A3[Cytoskeleton] is always True: True
A3[CTLA4] is always True: False
A3[CTLA4] is always False: False
A3[P2] is always False: True


First, note that the phenotype expression agrees with observations from \[4], and we can also confirm that `TCR` and `CTRA4` are oscillating within the unstable attractors, with `P2` expression differing between `A2` and `A3`. Furthermore, if we load this model into AEON, we can confirm this visually using the attractor state space explorer. In this visualisation, we can also easily observe that `P2` is in fact the only variable differentiating between `A2` and `A3`, and that the original observations from \[4] about stable values of several other network variables also hold (in particular, that `sFas = GPCR = S1P = SMAD = MCL1 = FLIP = IAP = true` in the complex attractors).

Next, we can analyse the basins of these three attractors. In \[4], this is only performed via sampling and simulation. Here, we can actually compute all strong and weak basins in full:


In [5]:
A1_weak = reach_bwd(graph, A1)

A2_weak = reach_bwd(graph, A2)
A3_weak = reach_bwd(graph, A3)

all_states_size = graph.unit_colored_vertices().vertices().cardinality()

A1_strong = A1_weak.minus(A2_weak).minus(A3_weak)
A1_strong_card = A1_strong.vertices().cardinality()
print("A1 strong basin:", (A1_strong_card / all_states_size) * 100.0, "%")
A1_realistic_strong = A1_strong.minus(apoptosis)
A1_realistic_strong_card = A1_realistic_strong.vertices().cardinality()
print("A1[Apoptosis = false] strong basin:", (A1_realistic_strong_card / all_states_size) * 100.0, "%")
A2_strong = A2_weak.minus(A1_weak).minus(A3_weak)
A2_strong_card = A2_strong.vertices().cardinality()
print("A2 strong basin:", (A2_strong_card / all_states_size) * 100.0, "%")
A3_strong = A3_weak.minus(A1_weak).minus(A2_weak)
A3_strong_card = A3_strong.vertices().cardinality()
print("A3 strong basin:", (A3_strong_card / all_states_size) * 100.0, "%")

A2_A3_strong = A2_weak.union(A3_weak).minus(A1_weak)
A2_A3_strong_card = A2_A3_strong.vertices().cardinality()
print("A2/A3 strong basin:", (A2_A3_strong_card / all_states_size) * 100.0, "%")

all_three = A1_weak.intersect(A2_weak).intersect(A3_weak)

print("Undetermined:", (all_three.vertices().cardinality() / all_states_size) * 100.0, "%")

all_phenotypes = A1_weak.intersect(A2_weak.union(A3_weak))
print("Undetermined phenotype:", (all_phenotypes.vertices().cardinality() / all_states_size) * 100.0, "%")

A1 strong basin: 50.04911422729492 %
A1[Apoptosis = false] strong basin: 0.049114227294921875 %
A2 strong basin: 1.9208528101444244e-07 %
A3 strong basin: 7.683411240577698e-07 %
A2/A3 strong basin: 3.073364496231079e-06 %
Undetermined: 37.26784971659072 %
Undetermined phenotype: 49.95088269934058 %


Just as postulated in \[4], the strong basin of `A1` represents slightly more than 50% of the state space, while the strong basins of other attractors are much smaller. Specifically, \[4] measured 0.24% and 0.03% for `A2` and `A3`. Meanwhile, our numbers are significantly lower -- however, the distribution of basin states is most likely different between the original and the reduced network. Note that the high percentage of states in the strong basin of `A1` is in fact not very surprising. Any state with `Apoptosis = true` will necessarily fall into this strong basin, which is exactly 50% of states. However, these states are not realistic as initial states. If we disregard such states, the basin is still the largest of the three by far (5 orders of magnitude), but the difference is reduced significantly.

Furthermore, we also compute the basin which guarantees that `A2` or `A3` is visited. Such a number is interesting since `A2` and `A3` differ only in the value of `P2`, which has little biological impact and we may thus consider them to be the manifestation of the same phenotype. In this case, we see that that the strong basin is larger than the basins of `A2` and `A3` in isolation, but is still much smaller than the basin of `A1` (4 orders of magnitude).

Finally, \[4] also mentions that a significant portion of states are completely undetermined, meaning that they can reach an arbitrary attractor. Here, this number can be calculated as the intersection of all weak basins, and is in fact quite high at 37%. Subsequently, we can also compute the number of states where the eventual phenotype of the network is undetermined at 49.95%. If we disregard the unrealistic initial states, these states amount to 74% and 99.9% of the network state space.

### Attractor Bifurcation

At this point, we have successfuly demonstrated that AEON can reliably replicate existing results of \[4], but with formal guarantees. However, the main advantage of AEON is the ability to explore multiple parametrisations of the same network simultaneously. Now, we are thus going to explore the fully parametrised network with its 6 logical parameters. Note that this number is not very high (i.e. AEON can typically handle networks with a much larger parameter space), but it serves as an easy to understand demonstration of its capabilities.

This fully parametrised network can be analysed by AEON in several seconds. Subsequently, we can construct a bifurcation decision tree which depicts the impact of logical parameters on the network attractors, such as the one below. Note that this tree is *mostly* automatically generated, however, a small portion of decision nodes was picked by hand to better visually distinguish the possible modes of behaviour. The highlighted node is the one representing the conditions studied in \[4].

<p align="center">
  <img src="./t-lgl-tree-full.png" width="800"/>
</p>

Notice that in this tree, the logical parameter `Stimuli2` generally introduces an additional (diseased) attractor. By inspecting the regulatory graph, we can see that `Stimuli2` only impacts `P2`, which we identified as the primary source of multistability in our comparison with \[4]. The reason is an autoregulation on `P2` which allows it to become bistable assuming `Stimuli2` is inactive. To study the effect of this autoregulation, we also consider a simlified version of the network where this autoregulation is removed (`t-lgl-p2.aeon`). As we can see in the resulting tree below, the spurious multistability has been mostly eliminated. Every leaf except for a single instance admits exactly one or two attractors (and as we later discuss, these differentiate between healthy and unhealthy phenotypes). In the single case where multistability is still admissible, we can observe (by inspecting attractor state space) that we have an additional diseased attractor which is otherwise identical to the healthy attractor, except for the value of `Apoptosis`.

<p align="center">
  <img src="./p2-tree-full.png" width="500"/>
</p>

#### Healthy phenotypes

An interesting observation we can make about the decision tree (either the original, or the one with reduced multistability) is that every leaf consisting of a single attractor only contains the healthy stable attractor with `Apoptosis = true`. This means that, assuming `CD45` and `IL15` are kept low, sufficient presence of `TAX`, `Stimuli` or `PDGF` guarantees that the cell correctly undergoes apoptosis. As we already discussed, `Stimuli` corresponds to successful detection of the targeted viral load, and as such it is not unreasonable to assume that it ensures conditions which activate `Apoptosis`. 

However, both `TAX` and `PDGF` have otherwise been identified as `Apoptosis` inhibitors (see \[3] and \[5]). In fact, \[3] identified `IL15` and `PDGF` as two substances whose combined activity is sufficient to represent any known leukemic deregulations. But, based on our findings, in the context of this network, in the absence of `CD45` and `IL15`, we postulate that these substances actually promote `Apoptosis`.

Finally, an additional interesting observation we can draw by further experimenting with the tree is that the activity of either `CD45` or `IL15` is in fact sufficient to ensure the presence of phenotypes with deregulated `Apoptosis`. No other logical parameter has this effect.

#### Absence of `Proliferation`

We can also notice that regardles of the setting of logical parameters, `Proliferation` always stabilises to `false` in every attractor. While this is expected in attractors where `Apoptosis` is active, it may not be the desired effect in deseased cells which remain alive indefinitely. However, if we look at the regulatory graph of the model, we can quickly identify why this is the case. Aside from `Apoptosis` (which inhibits every variable), `Proliferation` is activated by `STAT3` and inhibited by `P27`. However, `P27` is also only activated by `STAT3`, and thus, assuming `STAT3` stabilises, `P27` eventually stabilises with the same value. Consequently, once `STAT3` becomes active and promotes `Proliferation`, `P27` eventually becomes active as well, and inhibits `Proliferation`. As a result, `Proliferation` can always be active only during a temporary window where `STAT3` is active, but `P27` has not been activated yet.

It is unlikely that this is the desired behaviour in general, even though it may be useful in this particular case of a model describing this specific part of the pathway. Note that in the paper which originally introduces the model (i.e. \[3]), a larger set of variables and regulations is initially considered (albeit without specific update functions). Here, we actually see a more complex network surrounding `Proliferation`. In particular, `STAT3` is now inhibited by `P27`, which may lead to oscillations of these two variables, theoretically leading to an attractor with intermittent activity of `Proliferation`. Furthermore, `STAT3` does not influence `Proliferation` directly, but rather through the variable `CMYC` (which is not included in the final model). However, `CMYC` also depends on other network variables and thus can also activate `Proliferation` in the absence of `STAT3`. As future work, one may consider re-including these regulations in the network to better model the proliferation process.

To quickly demonstrate the abovementioned principle, let us consider a modified network where `STAT3` still negatively depends on `P27`, as was the case in the original network (see `t-lgl-proliferation.aeon`). Furthermore, let us set the update function of `STAT3` to `!Apoptosis & F(JAK, P27)`, where `F` is a fresh functional parameter, as we do not know the exact interaction between `JAK` and `P27`. Subsequently, using the stability analysis tool in the decision tree editor, we can see that indeed, an attractor with oscillating value of `Proliferation` is admissible. We can then devise the following decision tree fragment:

<p align="center">
  <img src="./proliferation-tree-fragment.png" width="400" />
</p>

In this tree, both leafs only contain attractors with oscillating `Proliferation` (aside from the `Apoptosis = true` stable state) and differ again only in the bistability of `P2` induced by `Stimuli2`. Meanwhile, the remaining (not expanded) nodes only contain attractors with `Proliferation` inactive. This leads us to the conclusion that using `!Apoptosis & (JAK | !P27)` as the update function for `STAT3` indeed leads to existence of attractors with Proliferation active. However, these attractors only appear in the presence of `CD45`.

### Discussion

In this short study, we have explored the dynamics of a T-lymphocyte survival pathway. The goal of this pathway is to describe the root causes of dysregulated apoptosis mechanism in individuals suffering from leukemia.

First, we have confirmed the findings of \[4] with regards to network attractors. These include the existence of two complex attractors corresponding to the diseased phenotype (`Apoptosis = false`), and one stable attractor corresponding to the healthy phenotype (`Apoptosis = true`). Furthermore, we have confirmed the stabilisation (or destabilisation) of other variables inside these attractors, as discussed in \[4]. However, compared to \[4], we were able to obtain these results using the whole original network, without performing any significant reductions on the regulatory graph. 

Subsequently, we have also analysed the basins of attraction for these three attractors. In \[4], these were only approximated using stochastic simulation. Using our exact method, we obtained results which numerically do not match those of \[4] (most likely due to the employed reduction and stochastic simulation), but confirm the general suggested trends. Specifically, we have confirmed that (after discounting the 50% of `Apoptosis = true` states, which are biologically unrealistic as initial states), the strong basin of the healthy phenotype is much larger than those of unhealthy phenotypes. Specifically, 5 orders of magnitued if we consider the two diseased attractors separately, and 4 orders of magnitued if considered together. However, these strong basins are very small in relative to the undetermined weak basins. In particular, 37% of the network states can reach any of the three attractors, and 49.95% can reach either phenotype. After discounting the `Apoptosis = true` states, this translates to 74% and 99.9% of realistic initial states.

We follow up on this analysis by further exploring the long-term behaviour of the network under variation of parameters. To this end, we have produced a complete bifurcation decision tree which captures the influence of logical parameters on the presence of attractors. Additionally, we have also presented a decision tree for a variant of the network with `P2` autoregulation disabled. This autoregulation is the root cause for the presence of multiple diseased attractors but has otherwise very little functionality. The tree of the reduced network demonstrates this, since while the multistability is eliminated (originally triggered by `Stimuli2`, which regulates `P2`), the effect of other network parameters is preserved.

Furthermore, by performing stability analysis on individual nodes of this tree, we have uncovered that in the absence of `CD45` and `IL15`, there are multiple logical parameters whose activity leads to the complete eradication of the diseased phenotype (in particular, `Stimuli`, `TAX` and `PDGF`). This is especially interesting due to the fact that in most other studies, `TAX` and `PDGF` are considered to be the drivers of apoptosis dysregulation. Aside from this effect, we have also identified `CD45` and `IL15` as two minimal sufficient conditions for the presence of diseased attractors: once either `CD45` or `IL15` is active, the network as is will always admit the diseased phenotype. Meanwhile, no other logical parameter has this property. However, `CD45` is one of the key proteins in T-lymphocyte antigen signalling \[6]. Meanwhile, `IL15` regulates the survival and proliferation of "memory" T-cells in the absence of antigens. Both are thus crucial to the proper functioning of lymphocytes as part of the immune system.

Finally, we have observed that the network as is only admits activity of `Proliferation` in temporary bursts outside of the network attractors. This may be a deficit in the model design, as monitoring of `Proliferation` was not the intended purpose of the network. Indeed, the regulatory mechanism of `Proliferation` is quite simplistic. We have consulted the original literature review in \[3] to identify a potential regulation (inhibition of `STAT3` by `P27`) which is omited in this network, but has the potential to influence the activity of `Proliferation` in network attractors. We have included this regulation into the model while leaving its influence unknown on the level of update functions using functional parameters. Subsequently, using a bifurcation decision tree, we have identified the conditions under which this regulation indeed leads to intermittent activity of `Proliferation` in the network attractors satisfying `Apoptosis = false`. Interestingly, activity of `CD45` is strictly necessary to produce this behaviour (`CD45 = false` always implies `Proliferation = false` in all attractors). This is in line with biological expectations, as `CD45` triggers response to antigens which may also involve T-cell proliferation \[7] (but is in general effected by different isoforms of `CD45` not included in this model).

### References

\[1] Sokol, Lubomir, and Thomas P. Loughran Jr. "Large granular lymphocyte leukemia." The oncologist 11.3 (2006): 263-273.

\[2] Shah, Mithun Vinod, Ranran Zhang, and Thomas P. Loughran Jr. "Never say die: survival signaling in large granular lymphocyte leukemia." Clinical Lymphoma and Myeloma 9 (2009): S244-S253.

\[3] Zhang, Ranran, et al. "Network model of survival signaling in large granular lymphocyte leukemia." Proceedings of the National Academy of Sciences 105.42 (2008): 16308-16313.

\[4] Saadatpour, Assieh, et al. "Dynamical and structural analysis of a T cell survival network identifies novel candidate therapeutic targets for large granular lymphocyte leukemia." PLoS computational biology 7.11 (2011): e1002267.

\[5] Okamoto, Kazuo, et al. "Human T‐cell leukemia virus type‐I oncoprotein Tax inhibits Fas‐mediated apoptosis by inducing cellular FLIP through activation of NF‐κB." Genes to Cells 11.2 (2006): 177-191.

\[6] Courtney, Adam H., et al. "CD45 functions as a signaling gatekeeper in T cells." Science signaling 12.604 (2019).

\[7] Ledbetter, Jeffrey A., et al. "CD4, CD8 and the role of CD45 in T-cell activation." Current opinion in immunology 5.3 (1993): 334-340.